In [1]:
import pandas as pd
import os
from sqlalchemy import create_engine


In [2]:
degreesTPB_df = pd.read_csv("degrees-that-pay-back.csv")
degreesTPB_df

,Undergraduate Major,Starting Median Salary,Mid-Career Median Salary,Percent change from Starting to Mid-Career Salary,Mid-Career 10th Percentile Salary,Mid-Career 25th Percentile Salary,Mid-Career 75th Percentile Salary,Mid-Career 90th Percentile Salary
0,Accounting,"$46,000.00","$77,100.00",67.6,"$42,200.00","$56,100.00","$108,000.00","$152,000.00"
1,Aerospace Engineering,"$57,700.00","$101,000.00",75.0,"$64,300.00","$82,100.00","$127,000.00","$161,000.00"
2,Agriculture,"$42,600.00","$71,900.00",68.8,"$36,300.00","$52,100.00","$96,300.00","$150,000.00"
3,Anthropology,"$36,800.00","$61,500.00",67.1,"$33,800.00","$45,500.00","$89,300.00","$138,000.00"
4,Architecture,"$41,600.00","$76,800.00",84.6,"$50,600.00","$62,200.00","$97,000.00","$136,000.00"
5,Art History,"$35,800.00","$64,900.00",81.3,"$28,800.00","$42,200.00","$87,400.00","$125,000.00"
6,Biology,"$38,800.00","$64,800.00",67.0,"$36,900.00","$47,400.00","$94,500.00","$135,000.00"
7,Business Management,"$43,000.00","$72,100.00",67.7,"$38,800.00","$51,500.00","$102,000.00","$147,000.00"
8,Chemical Engineering,"$63,200.00","$107,000.00",69.3,"$71,900.00","$87,300.00","$143,000.00","$194,000.00"
9,Chemistry,"$42,600.00","$79,900.00",87.6,"$45,300.00","$60,700.00","$108,000.00","$148,000.00"


In [3]:
degreesTPB_df = degreesTPB_df.rename(columns={"Undergraduate Major": "Undergraduate_Major"})
degreesTPB_df

,Undergraduate_Major,Starting Median Salary,Mid-Career Median Salary,Percent change from Starting to Mid-Career Salary,Mid-Career 10th Percentile Salary,Mid-Career 25th Percentile Salary,Mid-Career 75th Percentile Salary,Mid-Career 90th Percentile Salary
0,Accounting,"$46,000.00","$77,100.00",67.6,"$42,200.00","$56,100.00","$108,000.00","$152,000.00"
1,Aerospace Engineering,"$57,700.00","$101,000.00",75.0,"$64,300.00","$82,100.00","$127,000.00","$161,000.00"
2,Agriculture,"$42,600.00","$71,900.00",68.8,"$36,300.00","$52,100.00","$96,300.00","$150,000.00"
3,Anthropology,"$36,800.00","$61,500.00",67.1,"$33,800.00","$45,500.00","$89,300.00","$138,000.00"
4,Architecture,"$41,600.00","$76,800.00",84.6,"$50,600.00","$62,200.00","$97,000.00","$136,000.00"
5,Art History,"$35,800.00","$64,900.00",81.3,"$28,800.00","$42,200.00","$87,400.00","$125,000.00"
6,Biology,"$38,800.00","$64,800.00",67.0,"$36,900.00","$47,400.00","$94,500.00","$135,000.00"
7,Business Management,"$43,000.00","$72,100.00",67.7,"$38,800.00","$51,500.00","$102,000.00","$147,000.00"
8,Chemical Engineering,"$63,200.00","$107,000.00",69.3,"$71,900.00","$87,300.00","$143,000.00","$194,000.00"
9,Chemistry,"$42,600.00","$79,900.00",87.6,"$45,300.00","$60,700.00","$108,000.00","$148,000.00"


In [4]:
degreesTPB_df.Undergraduate_Major.unique()

array(['Accounting', 'Aerospace Engineering', 'Agriculture',
       'Anthropology', 'Architecture', 'Art History', 'Biology',
       'Business Management', 'Chemical Engineering', 'Chemistry',
       'Civil Engineering', 'Communications', 'Computer Engineering',
       'Computer Science', 'Construction', 'Criminal Justice', 'Drama',
       'Economics', 'Education', 'Electrical Engineering', 'English',
       'Film', 'Finance', 'Forestry', 'Geography', 'Geology',
       'Graphic Design', 'Health Care Administration', 'History',
       'Hospitality & Tourism', 'Industrial Engineering',
       'Information Technology (IT)', 'Interior Design',
       'International Relations', 'Journalism',
       'Management Information Systems (MIS)', 'Marketing', 'Math',
       'Mechanical Engineering', 'Music', 'Nursing', 'Nutrition',
       'Philosophy', 'Physician Assistant', 'Physics',
       'Political Science', 'Psychology', 'Religion', 'Sociology',
       'Spanish'], dtype=object)

In [5]:
for col in degreesTPB_df.columns:
    if 'Salary' in col:
        degreesTPB_df[col] = degreesTPB_df[col].replace( '[\$,)]','', regex=True ).astype(float)

In [6]:
degreesTPB_df = degreesTPB_df.drop(columns=['Percent change from Starting to Mid-Career Salary', 'Mid-Career 10th Percentile Salary', 'Mid-Career 25th Percentile Salary', 'Mid-Career 75th Percentile Salary', 'Mid-Career 90th Percentile Salary'])
degreesTPB_df

,Undergraduate_Major,Starting Median Salary,Mid-Career Median Salary
0,Accounting,46000.0,77100.0
1,Aerospace Engineering,57700.0,101000.0
2,Agriculture,42600.0,71900.0
3,Anthropology,36800.0,61500.0
4,Architecture,41600.0,76800.0
5,Art History,35800.0,64900.0
6,Biology,38800.0,64800.0
7,Business Management,43000.0,72100.0
8,Chemical Engineering,63200.0,107000.0
9,Chemistry,42600.0,79900.0


# Connect to local database

In [16]:
rds_connection_string = "postgres:postgres@localhost:5432/HigherEducation"
engine = create_engine(f'postgresql://{rds_connection_string}')

# Check For Tables

In [17]:
engine.table_names()

['salaries_per_major']

# Use pandas to load csv converted DataFrame into database

In [18]:
degreesTPB_df.head(2)

,Undergraduate_Major,Starting Median Salary,Mid-Career Median Salary
0,Accounting,46000.0,77100.0
1,Aerospace Engineering,57700.0,101000.0


In [19]:
if_exists_param = 'replace'

degreesTPB_df.to_sql(name='salaries_per_major', con=engine, if_exists=if_exists_param, index=False)

# Confirm data has been added by querying the HigherEducation table

In [21]:
pd.read_sql_query('select * from salaries_per_major', con=engine)#.head()

,Undergraduate_Major,Starting Median Salary,Mid-Career Median Salary
0,Accounting,46000.0,77100.0
1,Aerospace Engineering,57700.0,101000.0
2,Agriculture,42600.0,71900.0
3,Anthropology,36800.0,61500.0
4,Architecture,41600.0,76800.0
5,Art History,35800.0,64900.0
6,Biology,38800.0,64800.0
7,Business Management,43000.0,72100.0
8,Chemical Engineering,63200.0,107000.0
9,Chemistry,42600.0,79900.0
